### 텍스트 마이닝
- 텍스트로부터 유용한 인사이트를 발굴하는 과정 (의미있는 정보 추출)
### 자연어처리란?
- 자연어(인간이 일상생활에서 사용하는 언어)를 의미분석하여 처리
- ex) 음성인식, 챗봇

### Text Mining 종류
- 텍스트 요약 : 텍스트에서 중요한 주제를 나타내는 키워드를 추출, 문장생성
- 텍스트 분류 : 텍스트가 속한 카테고리를 분류
- 감성 분석 : 텍스트에서 나타나는 감정, 기분, 의견 분석
- 텍스트 군집화와 유사도 측정 : 텍스트의 비슷한 정도를 측정하고 그루핑해주는 분석

### Text Mining 과정
1. 텍스트 수집
  - crawling, download
2. 전처리
  - 용도에 맞게 텍스트를 처리하는 과정
    - 불용어제거 : 의미없는 단어를 제거(감탄사, 이모티콘 제거)
    - 정제, 정규화(좋다,짱이다 -> 좋다), 어간추출(먹다,먹어서,먹지-> 먹), 표제어 추출
    - 띄어쓰기, 오탈자 제거
3. 토큰화
  - 텍스트데이터를 잘게 쪼개주는 작업
  - n-gram: n개의 연속된 단어를 하나의 단어로 취급
4. 특징 추출
  - 중요단어를 선별하는 과정
  - BOW (Countvectorize, TF-IDF)
5. 데이터 분석
  - ML, DL 학습, 예측,평가

### 데이터 불러오기

In [6]:
import pandas as pd

In [ ]:
#컬럼 안의 글자 모두 확인
pd.set_option('display.max_colwidth', 100)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/ratings_train.txt', delimiter='\t')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/ratings_test.txt', delimiter='\t')

In [ ]:
train.head()
#id: 영화를 구분하기 위한 고유의 값(신경x)
#document : 리뷰
#label : 1(긍정), 0(부정)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#결측치 삭제
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
#데이터의 크기 확인
train.shape, test.shape

((149995, 3), (49997, 3))

### 단어의 빈도 분석
- 단어빈도는 문서의 요약, 분류 중요하게 사용된다.
- 워드카운트 : 단어의 등장 빈도를 측정하는 알고리즘
- 시각화 : 워드클라우드


In [ ]:
# 리뷰 데이터만 추출
text_train = train['document']

In [ ]:
# Series 형태
text_train
# 문장 -> 토큰화 진행

0                                                   아 더빙.. 진짜 짜증나네요 목소리
1                                     흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                                     너무재밓었다그래서보는것을추천한다
3                                         교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
                                      ...                              
149995                                              인간이 문제지.. 소는 뭔죄인가..
149996                                                    평점이 너무 낮아서...
149997                                  이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                                      청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                                         한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [ ]:
#토큰화 (띄어쓰기 단위로 모든 문장을 쪼개주자!)
temp = [doc.split(' ')for doc in text_train]
temp
# 이중리스트 > 1차원의 리스트로 담아주는 작업

[['아', '더빙..', '진짜', '짜증나네요', '목소리'],
 ['흠...포스터보고', '초딩영화줄....오버연기조차', '가볍지', '않구나'],
 ['너무재밓었다그래서보는것을추천한다'],
 ['교도소', '이야기구먼', '..솔직히', '재미는', '없다..평점', '조정'],
 ['사이몬페그의',
  '익살스런',
  '연기가',
  '돋보였던',
  '영화!스파이더맨에서',
  '늙어보이기만',
  '했던',
  '커스틴',
  '던스트가',
  '너무나도',
  '이뻐보였다'],
 ['막', '걸음마', '뗀', '3세부터', '초등학교', '1학년생인', '8살용영화.ㅋㅋㅋ...별반개도', '아까움.'],
 ['원작의', '긴장감을', '제대로', '살려내지못했다.'],
 ['별',
  '반개도',
  '아깝다',
  '욕나온다',
  '이응경',
  '길용우',
  '연기생활이몇년인지..정말',
  '발로해도',
  '그것보단',
  '낫겟다',
  '납치.감금만반복반복..이드라마는',
  '가족도없다',
  '연기못하는사람만모엿네'],
 ['액션이', '없는데도', '재미', '있는', '몇안되는', '영화'],
 ['왜케', '평점이', '낮은건데?', '꽤', '볼만한데..', '헐리우드식', '화려함에만', '너무', '길들여져', '있나?'],
 ['걍인피니트가짱이다.진짜짱이다♥'],
 ['볼때마다', '눈물나서', '죽겠다90년대의', '향수자극!!허진호는', '감성절제멜로의', '달인이다~'],
 ['울면서', '손들고', '횡단보도', '건널때', '뛰쳐나올뻔', '이범수', '연기', '드럽게못해'],
 ['담백하고',
  '깔끔해서',
  '좋다.',
  '신문기사로만',
  '보다',
  '보면',
  '자꾸',
  '잊어버린다.',
  '그들도',
  '사람이었다는',
  '것을.'],
 ['취향은',
  '존중한다지만',
  '진짜',
  '내생에',
  '극장에서',
  '본',
  '영화중',
  '가장',
  '

In [ ]:
token_list = []
for i in temp :
  token_list += i

In [ ]:
token_list

['아',
 '더빙..',
 '진짜',
 '짜증나네요',
 '목소리',
 '흠...포스터보고',
 '초딩영화줄....오버연기조차',
 '가볍지',
 '않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소',
 '이야기구먼',
 '..솔직히',
 '재미는',
 '없다..평점',
 '조정',
 '사이몬페그의',
 '익살스런',
 '연기가',
 '돋보였던',
 '영화!스파이더맨에서',
 '늙어보이기만',
 '했던',
 '커스틴',
 '던스트가',
 '너무나도',
 '이뻐보였다',
 '막',
 '걸음마',
 '뗀',
 '3세부터',
 '초등학교',
 '1학년생인',
 '8살용영화.ㅋㅋㅋ...별반개도',
 '아까움.',
 '원작의',
 '긴장감을',
 '제대로',
 '살려내지못했다.',
 '별',
 '반개도',
 '아깝다',
 '욕나온다',
 '이응경',
 '길용우',
 '연기생활이몇년인지..정말',
 '발로해도',
 '그것보단',
 '낫겟다',
 '납치.감금만반복반복..이드라마는',
 '가족도없다',
 '연기못하는사람만모엿네',
 '액션이',
 '없는데도',
 '재미',
 '있는',
 '몇안되는',
 '영화',
 '왜케',
 '평점이',
 '낮은건데?',
 '꽤',
 '볼만한데..',
 '헐리우드식',
 '화려함에만',
 '너무',
 '길들여져',
 '있나?',
 '걍인피니트가짱이다.진짜짱이다♥',
 '볼때마다',
 '눈물나서',
 '죽겠다90년대의',
 '향수자극!!허진호는',
 '감성절제멜로의',
 '달인이다~',
 '울면서',
 '손들고',
 '횡단보도',
 '건널때',
 '뛰쳐나올뻔',
 '이범수',
 '연기',
 '드럽게못해',
 '담백하고',
 '깔끔해서',
 '좋다.',
 '신문기사로만',
 '보다',
 '보면',
 '자꾸',
 '잊어버린다.',
 '그들도',
 '사람이었다는',
 '것을.',
 '취향은',
 '존중한다지만',
 '진짜',
 '내생에',
 '극장에서',
 '본',
 '영화중',
 '가장',
 '노잼',
 '노감동임',
 '스토리도'

### Counter 활용 단어 빈도 측정


In [ ]:
from collections import Counter

In [ ]:
cnt = Counter(token_list)

In [ ]:
#빈도수 내림차순 출력
cnt.most_common(10)
#출력하고 싶은 데이터 개수

[('영화', 10825),
 ('너무', 8239),
 ('정말', 7791),
 ('진짜', 5929),
 ('이', 5059),
 ('영화.', 3598),
 ('왜', 3285),
 ('더', 3260),
 ('이런', 3249),
 ('그냥', 3237)]

### 워드 클라우드 만들기~

In [ ]:
!pip install wordcloud


SyntaxError: ignored

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud(background_color = 'white',
          font_path="/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf",
          random_state=1013)

In [ ]:
# 나눔 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#WordCloud 냅적으로 토큰화 기능을 가지고 있음 > 하나의 문자열로 만들어주기!
one_str = " ".join(token_list)
#WordCloud 그림 만들기 (토큰화 후 빈도수를 세고 그림을 그려준다!)
cloud = wc.generate_from_text(one_str)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,795 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120875 files and dire

In [ ]:
#시각화
import matplotlib.pyplot as plt

In [ ]:
#한글 데이터 출력을 위한 준비
plt.rc('font',family='NanumBarenGothic')

In [ ]:
plt.figure(figsize=(20,8))
plt.imshow(cloud)
plt.axis('off') #X축,Y축 제거
plt.show()

NameError: ignored

<Figure size 2000x800 with 0 Axes>

### 여러가지의 형태소 분석기 사용해보기~
- 한국어는 교착어로 어근+접사 > 자연어 처리가 어려운 특징
- 우리가 분석하고자 하는 데이터에 따라서 다른 형태소 분석기를 사용

- Konlpy : 한국어를 위한 형태소 분석기 (한국어 처리를 위한 파이썬 패키지)


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 52.4 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Kkma #서울대에서 만든 한국어 형태소 처리기
from konlpy.tag import Okt #트위터에서 만든 한국어 형태소 처리기

In [ ]:
# 형태소 분석기 생성
kkm=Kkma()
okt=Okt()

In [ ]:
# 명사 추출
kkm.nouns("아버지가 방에 들어가신다")

['아버지', '방']

In [ ]:
kkm.nouns("아버지가방에들어가신다")

['아버지', '아버지가방', '가방']

In [ ]:
okt.nouns('아버지가 방에 들어가신다')

['아버지', '방']

In [ ]:
okt.nouns('아버지가방에들어가신다')

['아버지', '가방']

In [ ]:
#품사 추출 -> 문장을 형태소 단위로 잘라서 품사를 분류
kkm.morphs("아버지가 방에 들어가신다")

['아버지', '가', '방', '에', '들어가', '시', 'ㄴ다']

In [ ]:
okt.morphs('아버지가 방에 들어가신다')

['아버지', '가', '방', '에', '들어가신다']

In [ ]:
#각각의 형태소가 어떤 품사를 가지는지 확인
kkm.pos('아버지가 방에 들어가신다')

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN')]

In [ ]:
#품사명 확인
kkm.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [ ]:
len(okt.tagset)

19

## 머신러닝기반 감정분석
프로세스 : 수집 -> (정제) -> 토큰화 -> 수치화 -> 학습 -> 튜닝 -> 평가

### 감성분석
1.(전통적) 감성사진을 이용한 분석

2.머신러닝 기반의 감성분석

### 토큰화 & 수치화
- BOW (Bag of word)
 - 단점 : 문맥의 순서를 고려하지 않아 자연어처리에 부적합


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer #Bow를 구현한 클래스

In [ ]:
cv_test = CountVectorizer()

In [ ]:
temp = ['안녕 나는 바나나야',
        '오늘 점심을 무엇을 먹을까?',
        '그 영화는 너무 재미가 없어',
        '오늘 날씨가 추운데 외투입고 가 ']

In [ ]:
# 토큰화 및 단어사전 구축
# 기본적으로 띄어쓰기 기준으로 토큰화
# 영어 기본적으로 대문자 -> 소문자로 변경
# 기본적으로 문장부호나 한글자는 없음(한글자 단어는 파라미터로 변경가능)
cv_test.fit(temp)

CountVectorizer()

In [ ]:
len(cv_test.vocabulary_) # 구축된 단어사전의 길이 확인

14

In [ ]:
print(cv_test.vocabulary_)

{'안녕': 6, '나는': 0, '바나나야': 5, '오늘': 9, '점심을': 12, '무엇을': 4, '먹을까': 3, '영화는': 8, '너무': 2, '재미가': 11, '없어': 7, '날씨가': 1, '추운데': 13, '외투입고': 10}


In [ ]:
# 수치화
temp_transformed = cv_test.transform(temp)


In [ ]:
# 0이라는 데이터가 불필요하게 많기 때문에
# 압축된 희소행렬 형태로 저장
# 원본은 toarray() 함수 활용
temp_transformed

<4x14 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

## ML모델 학습

In [ ]:
# 학습데이터 셋 구성
naver_movie_cv = CountVectorizer() # BOW를 진행하는 객체 생성
naver_movie_cv.fit(train['document']) # 토큰화 및 단어사전 구축
len(naver_movie_cv.vocabulary_) # 단어사전 길이 확인 293366

293366

In [ ]:
X_train = naver_movie_cv.transform(train['document'])
X_test = naver_movie_cv.transform(test['document'])

In [ ]:
# 정답 데이터 구성 (1은 긍정리뷰 0은 부정리뷰)
y_train = train['label']
y_test = test['label']

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(149995, 293366) (149995,)
(49997, 293366) (49997,)


In [ ]:
# 분류 모델활용 -> 로지스틱 모델
# 교차검증으로 검수

In [100]:
# 테스트 데이터 활용 펑가
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
# 로지스틱 회귀 모델 생성
model = LogisticRegression()


In [ ]:
# 5-겹 교차 검증 수행 (k=5)
scores = cross_val_score(model, X_train, y_train, cv=3)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
model.score(X_test,y_test)

0.8142488549312958

In [ ]:
# 모델 활용
sample = naver_movie_cv.transform(['어제 본 영화는 너무 재미가 없었어',
                                   '팝콘 먹으면서 보러 가는거지',
                                   '언제 집에 가지?',
                                   '너무 재밌음']
                                )
model.predict(sample)

array([0, 0, 0, 1])

In [ ]:
model.predict_proba(sample) # 확률 값으로 확인

array([[0.87775068, 0.12224932],
       [0.74053479, 0.25946521],
       [0.5593182 , 0.4406818 ],
       [0.06422218, 0.93577782]])

### 학습된 토큰(단어) 가중치 확인
- 긍정리뷰에 많이 등장하는 단어는 가중치가 양수 값으로 커짐
- 부정리뷰에 많이 등장하는 단어는 가중치가 음수 값으로 작아짐

In [ ]:
# 학습된 가중치 갯수 확인
naver_movie_coef = model.coef_[0]
len(naver_movie_coef)

293366

In [ ]:
naver_movie_coef

array([ 0.52720314,  0.00189109, -0.35608771, ..., -0.14786802,
       -0.31474984, -0.35608771])

In [ ]:
# vocab 정렬하기
import pandas as pd

In [ ]:
df = pd.DataFrame([naver_movie_cv.vocabulary_.keys(),
                   naver_movie_cv.vocabulary_.values()],
                  index=['단어','인덱스'])

In [ ]:
df = df.T

In [ ]:
df

In [ ]:
df.sort_values(by='인덱스', inplace=True)

In [ ]:
df['가중치'] = naver_movie_coef # 가중치와 vacab 연결


In [ ]:
df.head()

,단어,인덱스,가중치
126724,00,0,0.527203
85452,000,1,0.001891
123217,0000000000000000,2,-0.356088
245647,00000000000000000000000000000을달라,3,-0.356088
55188,000000000000000001개짜리,4,-0.147074


In [ ]:
# 가중치 중심으로 정렬
df.sort_values(by='가중치', inplace=True)

In [ ]:
# 긍정 단어토큰 상위 20개
df.tail(20)

In [ ]:
# 부정 단어토큰 하위 20개
df.head(20)

,단어,인덱스,가중치
1982,최악의,256927,-4.445200
1115,최악,256888,-3.871064
3490,쓰레기영화,159352,-3.739387
554,졸작,235972,-3.545920
161,재미없다,222651,-3.396115
192,지루하다,243852,-3.291784
2040,최악이다,256971,-3.233242
3955,1점도,2597,-3.158111
104,노잼,59668,-3.123785
2946,재미없어,222746,-3.097590


### tf-idf 활용하기
 - tf : 개별문서에 특정단어의 등장 빈도
  - 특정단어의 빈도가 높으면 문서를 대표하는 특징이 된다.
 - df : 특정단어를 가지는 말뭉치 내 문서의 갯수  
  - 전체 말뭉치에 너무 과도하게 등장하면 특징으로서의 의미가 사라진다.
 - tf-idf = td*(1/df)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [65]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 40.7 MB/s eta 0:00:00


In [5]:
# konlpy 형태소 단위로 분리
from konlpy.tag import Okt
from tqdm import tqdm

ModuleNotFoundError: ignored

In [ ]:
train_morphs = [] # 형태소 단위로 쪼개진 리뷰를 담을 리스트 생성
for doc in tqdm(train['document']): # 약 15만개의 리뷰 반복
  morphs_rs = okt.morphs(doc) # 형태소 단위로 분리
  train_morphs.append(" ".join(morphs_rs)) # tfidfvectorizer가 띄어쓰기 중심으로 토큰화하도록 기능이 내장되어 있어 한문장으로 만듬


100%|██████████| 149995/149995 [06:34<00:00, 380.54it/s]


In [ ]:
# 형태소로 쪼개진 리스트를 pickle로 저장
import pickle # 파이썬에 존재하는 모든 데이터 타입을 파일로 저장

In [ ]:
# 오픈할 파일명, 모드(쓰기모드)
with open('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/X_train_morphs.pkl','wb') as f :
  pickle.dump( train_morphs, f)# 파일 저장

In [ ]:
# 오픈할 파일명, 모드(읽기모드)
with open('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/X_train_morphs.pkl','rb') as f :
  X_train_morphs = pickle.load(f) # 파일 로딩

In [ ]:
# 1.tfidf활용 토큰화 및 수치화
naver_movie_tfidf = TfidfVectorizer() # tfidf 객체 생성
naver_movie_tfidf.fit(X_train_morphs) # 토큰화 및 단어사전 구축
display(len(naver_movie_tfidf.vocabulary_)) # 단어사전의 길이 확인
tfidf_transformed = naver_movie_tfidf.transform(X_train_morphs) # 수치화


99926

In [ ]:
# 2.로지스틱 모델활용 교차검증
m2 = LogisticRegression()
score = cross_val_score(m2, tfidf_transformed, y_train, cv = 3)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
score


array([0.83881678, 0.83363335, 0.83373335])

In [ ]:
# 3.테스트 데이타 활용 평가
# - 테스트 데이터 형태소 분석기 돌리기
test_morphs = [] # 형태소 단위로 쪼개진 리뷰를 담을 리스트 생성
for doc in tqdm(test['document']): # 약 15만개의 리뷰 반복
  morphs_rs = okt.morphs(doc) # 형태소 단위로 분리
  test_morphs.append(" ".join(morphs_rs)) # tfidfvectorizer가 띄어쓰기 중심으로 토큰화하도록 기능이 내장되어 있어 한문장으로 만듬


100%|██████████| 49997/49997 [03:11<00:00, 261.14it/s]


In [ ]:
# - 토큰화 및 수치화
tfidf_transformed_test = naver_movie_tfidf.transform(test_morphs) # 수치화
# - score 함수 활용 평가
m2.fit(tfidf_transformed, y_train)
m2.score(tfidf_transformed_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8403104186251175

## 혐오표현 분류기

In [12]:
%cd '/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/'

/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning


In [138]:
hate_train = pd.read_csv('./data/unsmile_train_v1.0.tsv', delimiter='\t')
hate_valid = pd.read_csv('./data/unsmile_valid_v1.0.tsv', delimiter='\t')

In [139]:
hate_train.head()

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0


In [140]:
# 컬럼 label
# 혐오표현 0 / 악플,욕설 1 / clean 2
hate_train['label'] = 0
# 전처리
hate_train.head()

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭,label
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0,0


In [141]:
hate_train.drop([5876,11942], inplace= True)

In [143]:
hate_train['혐오표현'] = hate_train.loc[:,'여성/가족':'기타 혐오'].sum(axis=1)


0        0
1        1
2        0
3        0
4        1
        ..
15000    0
15001    1
15002    2
15003    0
15004    1
Name: 혐오표현, Length: 15003, dtype: int64

In [149]:
def labeling(row) :
  if row['혐오표현'] > 0 :
    return 0
  elif row['악플/욕설'] > 0 :
    return 1
  elif row['clean'] > 0 :
    return 2


In [150]:
hate_train['label'] = hate_train[['혐오표현','악플/욕설','clean']].apply(labeling,axis = 1)

In [151]:
hate_train.head()

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭,label,혐오표현
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0,2,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0,0,1
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0,2,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0,2,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0,0,1


## 감성분석 실습

In [152]:
# 감성분석 실습
# 1. X는 문장컬럼, y는 label 컬럼활용

X = hate_train['문장']
y = hate_train['label']



y

0        2
1        0
2        2
3        2
4        0
        ..
15000    2
15001    0
15002    0
15003    1
15004    0
Name: label, Length: 15003, dtype: int64

In [153]:
# 2. konlpy를 사용해서 형태소 추출(선택)
# konlpy 형태소 단위로 분리
from konlpy.tag import Okt
from tqdm import tqdm
okt = Okt() # 형태소 분석기 생성
train_morphs = [] # 형태소 단위로 쪼개진 리뷰를 담을 리스트 생성
for doc in tqdm(X): # 약 15000개 리뷰 반복
  morphs_rs = okt.morphs(doc) # 형태소 단위로 분리
  train_morphs.append(" ".join(morphs_rs))

100%|██████████| 15003/15003 [00:51<00:00, 293.36it/s]


In [154]:
# 3. Bow or Tfidf (선택) -> 토큰화 및 수치화
hate_tfidf = TfidfVectorizer(stop_words=['은','는','이','가'],
                             min_df = 3, max_df = 1000) # tfidf 객체 생성
hate_tfidf.fit(train_morphs) # 토큰화 및 단어사전 구축
print('단어길이',len(hate_tfidf.vocabulary_)) # 단어사전의 길이 확인
X_train = hate_tfidf.transform(train_morphs) # 수치화
print(X_train.shape)


단어길이 7618
(15003, 7618)


In [155]:

# 4. 선형분류 모델활용 학습 및 평가 -> 교차검증 활용
hate_logi = LogisticRegression()
hate_logi


LogisticRegression()

In [156]:
score = cross_val_score(hate_logi, X_train, y, cv=3 )

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [159]:
# 5. 테스트 데이터셋 활용 fianl 평가 확인


hate_valid['혐오표현'] = hate_valid.loc[:,'여성/가족':'기타 혐오'].sum(axis=1)
hate_valid['label'] = hate_valid[['혐오표현','악플/욕설','clean']].apply(labeling,axis = 1)

X_val = hate_valid['문장']
y_val = hate_valid['label']

test_morphs = [] # 형태소 단위로 쪼개진 리뷰를 담을 리스트 생성
for doc in tqdm(X_val): # 약 15000개 리뷰 반복
  morphs_rs = okt.morphs(doc) # 형태소 단위로 분리
  test_morphs.append(" ".join(morphs_rs))

if test_morphs:
    X_val = hate_tfidf.transform(test_morphs)
else:
    print("No data available for transformation.")



100%|██████████| 3737/3737 [00:15<00:00, 236.71it/s]


In [160]:
hate_logi.fit(X_train,y )
hate_logi.score(X_val, y_val)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6997591651056998

In [162]:
# 분류평가지표 확인
from sklearn.metrics import classification_report

In [165]:
pre_val = hate_logi.predict(X_val)
print(classification_report(y_val, pre_val))

              precision    recall  f1-score   support

           0       0.75      0.89      0.82      2016
           1       0.60      0.35      0.44       786
           2       0.61      0.58      0.59       935

    accuracy                           0.70      3737
   macro avg       0.65      0.61      0.62      3737
weighted avg       0.68      0.70      0.68      3737



In [170]:
# 0번 클래스(혐오표현) 가중치 확인하기
weight_class_0 = hate_logi.coef_[0]
len(weight_class_0)

7618

In [172]:
# 단어사전 꺼내오기
hate_vocab = hate_tfidf.vocabulary_
len(hate_vocab)

7618

In [175]:
# df생성
hate_df = pd.DataFrame([hate_vocab.keys(), hate_vocab.values()],
                       index=['단어','인덱스']).T

In [206]:
hate_df.sort_values(by='인덱스', inplace=True)
hate_df['혐오표현 가중치'] = weight_class_0
hate_df

,단어,인덱스,악플/욕설 가중치,혐오표현 가중치
12,10,0,-0.077295,-0.061937
2687,100,1,0.278958,0.567742
6469,100년,2,-0.078802,-0.107878
2202,10년,3,-0.626746,-0.302596
7464,10만,4,-0.195082,-0.330223
...,...,...,...,...
2018,힘들다,7613,0.205064,0.330785
7035,힘들다고,7614,-0.208749,0.353915
5651,힘들어,7615,-0.332620,-0.194086
7520,힘들어서,7616,0.329640,-0.067663


In [181]:
# 악플/욕설 클래스(혐오표현) 가중치 확인하기
weight_class_1 = hate_logi.coef_[1]
len(weight_class_1)

7618

In [207]:
# hate_df.drop(labels='혐오표현 가중치', axis=1, inplace=True)
# hate_df.drop(labels='혐오표현 가중치1', axis=1, inplace=True)
# hate_df.drop(labels='악플/욕설 가중치', axis=1, inplace=True)


In [208]:
hate_df.sort_values(by='인덱스', inplace=True)
hate_df['악플/욕설 가중치'] = weight_class_1
hate_df.sort_values(by='악플/욕설 가중치', ascending=False).head(20)

,단어,인덱스,혐오표현 가중치,악플/욕설 가중치
129,씨발,4229,1.173087,3.027529
622,시발,3975,0.839707,2.935237
78,병신,3065,0.996300,2.633295
1901,ㅅㅂ,158,0.366119,2.417841
448,지랄,6328,0.740557,2.160238
179,개돼지,457,-0.243040,1.824294
740,좆팔,6111,0.274706,1.748180
1103,싶다,4146,-0.650011,1.366575
975,인간,5423,0.030694,1.332348
88,대가리,1624,0.035610,1.316967
